<a href="https://colab.research.google.com/github/e19166/e19-4yp-Dynamic-Multi-Dimensional-Resource-Orchestration-in-Kubernetes/blob/main/results/hashgen_memory_limit_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('hashgen_memory_limit_reduction.csv')
df.head()  # to preview the data

,2025-05-12T00:24:38.863000,hash-gen-deployment,0.1,134217728,0.2,268435456,0.020389652840088056,0.035586941535081025,82288640
0,2025-05-12T00:25:08.947000,hash-gen-deployment,0.1,134217728,0.2,268435456,0.020390,0.040888,82288640
1,2025-05-12T00:25:39.020000,hash-gen-deployment,0.1,134217728,0.2,268435456,0.020884,0.042276,82288640
2,2025-05-12T00:26:09.086000,hash-gen-deployment,0.1,134217728,0.2,268435456,0.020361,0.040294,82288640
3,2025-05-12T00:26:39.147000,hash-gen-deployment,0.1,134217728,0.2,268435456,0.020754,0.041202,82288640
4,2025-05-12T00:27:09.209000,hash-gen-deployment,0.1,134217728,0.2,268435456,0.020476,0.040154,82288640
